<a href="https://colab.research.google.com/github/ByungjunKim/WikisourceParsing/blob/main/wikisource_tutorial_colab_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 위키문헌 파서 튜토리얼

**한국어 위키문헌에서 완전한 메타데이터를 추출하는 방법을 배워보세요!**

## 🎯 학습 목표
- 위키문헌 XML 덤프에서 텍스트와 메타데이터 추출하기
- API를 활용해 누락된 분류 정보 보강하기
- 위키데이터와 연동해 연도 정보 추출하기
- 실제 사용 가능한 깔끔한 텍스트 데이터 생성하기

## 📖 예제: 애국가 완전 분석
- **문제**: XML 덤프에서는 2개 분류만 나오는데, 실제 웹사이트에는 4개가 있음
- **해결**: API 보강으로 '1941년 작품', 'PD-old-50' 등 누락된 분류까지 완전 추출
- **결과**: 분류에서 연도 정보(1941년)까지 정확하게 추출 성공!

---
**✅ 노트북 테스트 완료**: 2025-09-11 10:21:57
- 모든 라이브러리 로딩 성공
- API 함수들 정상 작동
- 덤프 파일 확인 완료 (147.8MB)
- 멀티프로세싱 환경 준비 (11개 코어)

**🚀 실행 준비 완료!**

---
## 🎉 **최종 버전 - Colab 실행 준비 완료!**
**버전**: v1.0 Final | **생성일**: 2025-09-11 10:58:55

### ✅ **검증 완료 사항:**
- 모든 라이브러리 호환성 확인
- TypeError 및 정렬 오류 수정
- API 함수 정상 작동 테스트  
- 멀티프로세싱 환경 최적화
- 전체 위키문헌 파싱 기능 검증

### 🚀 **바로 실행 가능:**
1. Google Colab에 업로드
2. 런타임 연결 후 순차 실행
3. 애국가 분석부터 전체 파싱까지 단계별 학습

**💡 권장 실행 순서**: 기본 파싱 → API 보강 → 대량 처리 → 전체 파싱


## 🔧 환경 설정

먼저 필요한 라이브러리를 설치하고 임포트합니다.

In [1]:
# 필요한 라이브러리 설치
!pip install mwxml requests tqdm pandas

# 기본 라이브러리 임포트
import mwxml
import bz2
import json
import os
import re
import urllib.parse
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
from tqdm import tqdm
import pandas as pd
import requests

print("✅ 모든 라이브러리가 성공적으로 로드되었습니다!")
print(f"🖥️  사용 가능한 CPU 코어: {mp.cpu_count()}개")

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d05a2c530570b21cbdc935cfe7f27da781371df35b63c03b0b7c36486c0f9586
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
✅ 모든 라이브러리가 성공적으로 로드되었습니다!
🖥️  사용 가능한 CPU 코어: 2개


## 📥 데이터 다운로드

한국어 위키문헌 덤프 파일을 다운로드합니다.

In [2]:
# 위키문헌 덤프 다운로드
dump_url = "https://dumps.wikimedia.org/kowikisource/20250901/kowikisource-20250901-pages-articles.xml.bz2"
dump_file = "kowikisource-20250901-pages-articles.xml.bz2"

if not os.path.exists(dump_file):
    print("📥 위키문헌 덤프 다운로드 중... (약 150MB, 시간이 걸릴 수 있습니다)")
    !wget -O {dump_file} {dump_url}
    print("✅ 다운로드 완료!")
else:
    print("✅ 덤프 파일이 이미 존재합니다!")

# 파일 크기 확인
file_size = os.path.getsize(dump_file) / (1024 * 1024)  # MB
print(f"📁 파일 크기: {file_size:.1f} MB")

📥 위키문헌 덤프 다운로드 중... (약 150MB, 시간이 걸릴 수 있습니다)
--2025-09-11 02:40:22--  https://dumps.wikimedia.org/kowikisource/20250901/kowikisource-20250901-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154956955 (148M) [application/octet-stream]
Saving to: ‘kowikisource-20250901-pages-articles.xml.bz2’

kowikisource-202509 100%[===================>] 147.78M  4.23MB/s    in 35s     

2025-09-11 02:40:58 (4.22 MB/s) - ‘kowikisource-20250901-pages-articles.xml.bz2’ saved [154956955/154956955]

✅ 다운로드 완료!
📁 파일 크기: 147.8 MB


## 🌐 API 보강 함수들

위키문헌 API와 위키데이터 API를 사용해서 누락된 정보를 보강하는 함수들을 정의합니다.

In [3]:
def get_categories_from_api(page_title):
    """
    위키문헌 API에서 페이지의 모든 분류를 가져옵니다

    이 함수가 중요한 이유:
    - XML 덤프에는 기본 분류만 있음
    - 템플릿에서 자동 생성되는 분류는 API로만 확인 가능
    - 예: '1941년 작품', 'PD-old-50' 등
    """
    try:
        api_url = "https://ko.wikisource.org/w/api.php"
        params = {
            'action': 'query',
            'format': 'json',
            'titles': page_title,
            'prop': 'categories',
            'cllimit': 'max'
        }

        headers = {'User-Agent': 'WikisourceParser/1.0 (Educational Tutorial)'}
        response = requests.get(api_url, params=params, headers=headers, timeout=10)
        data = response.json()

        pages = data.get('query', {}).get('pages', {})
        page_id = list(pages.keys())[0]

        if page_id == '-1':
            return []

        categories = pages[page_id].get('categories', [])
        category_names = []

        for cat in categories:
            cat_title = cat['title']
            if cat_title.startswith('분류:'):
                category_names.append(cat_title[3:])  # '분류:' 제거

        return category_names

    except Exception as e:
        print(f"API 분류 조회 오류 ({page_title}): {e}")
        return []

def get_year_from_wikidata(page_title):
    """
    위키데이터에서 작품의 발표 연도를 가져옵니다

    위키데이터 연동의 장점:
    - 구조화된 연도 정보 제공
    - 여러 언어판에서 공유되는 정확한 데이터
    - 분류 정보와 교차 검증 가능
    """
    try:
        # 1단계: 위키문헌 페이지에서 위키데이터 ID 가져오기
        wikisource_api = "https://ko.wikisource.org/w/api.php"
        params = {
            'action': 'query',
            'format': 'json',
            'titles': page_title,
            'prop': 'pageprops'
        }

        headers = {'User-Agent': 'WikisourceParser/1.0 (Educational Tutorial)'}
        response = requests.get(wikisource_api, params=params, headers=headers, timeout=10)
        data = response.json()

        pages = data.get('query', {}).get('pages', {})
        page_id = list(pages.keys())[0]

        if page_id == '-1':
            return None

        wikidata_id = pages[page_id].get('pageprops', {}).get('wikibase_item')

        if not wikidata_id:
            return None

        # 2단계: 위키데이터에서 발표일 정보 가져오기
        wikidata_api = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbgetentities',
            'format': 'json',
            'ids': wikidata_id,
            'props': 'claims'
        }

        response = requests.get(wikidata_api, params=params, headers=headers, timeout=10)
        data = response.json()

        entity = data.get('entities', {}).get(wikidata_id, {})
        claims = entity.get('claims', {})

        # 발표 관련 속성들 확인
        date_properties = ['P577', 'P571', 'P585']  # 발표일, 시작일, 특정시점

        for prop in date_properties:
            if prop in claims:
                for claim in claims[prop]:
                    try:
                        time_value = claim['mainsnak']['datavalue']['value']['time']
                        # +1941-00-00T00:00:00Z 형태에서 연도 추출
                        year = int(time_value[1:5])
                        if 1800 <= year <= 2030:  # 유효한 연도 범위
                            return year
                    except (KeyError, ValueError):
                        continue

        return None

    except Exception as e:
        print(f"위키데이터 조회 오류 ({page_title}): {e}")
        return None

print("✅ API 보강 함수들이 정의되었습니다!")

✅ API 보강 함수들이 정의되었습니다!


## 🔍 텍스트 파싱 함수들

위키텍스트에서 메타데이터를 추출하고 깔끔한 본문을 만드는 함수들입니다.

In [4]:
def extract_metadata(text):
    """
    위키텍스트에서 메타데이터를 추출합니다

    추출하는 정보:
    - 저자: [[저자:이름]] 패턴에서
    - 분류: [[분류:이름]] 패턴에서
    - 작곡가: '작곡' 키워드 주변에서
    - 라이선스: {{PD-*}} 템플릿에서
    - 언어: '한자', '한글' 키워드에서
    """
    if not text:
        return {
            'authors': [],
            'categories': [],
            'composer': None,
            'translator': None,
            'year': None,
            'license': None,
            'language': None
        }

    # 저자 정보 추출
    authors = []
    author_patterns = [
        r'\[\[저자:([^|\]]+)',  # [[저자:이름]] 또는 [[저자:이름|표시명]]
        r'\|\s*author\s*=\s*\[\[저자:([^|\]]+)',  # author= 매개변수
    ]

    for pattern in author_patterns:
        matches = re.findall(pattern, text)
        authors.extend(matches)

    # 분류 정보 추출
    categories = re.findall(r'\[\[분류:([^\]]+)\]\]', text)

    # 작곡가 정보 추출
    composer = None
    composer_patterns = [
        r'([^.]+)\s*작곡',
        r'작곡가?\s*[:=]\s*([^.\n]+)'
    ]
    for pattern in composer_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            composer = matches[0].strip()
            break

    # 연도 정보 추출
    year_matches = re.findall(r'(\d{4})년', text)
    year = None
    if year_matches:
        year = max(set(year_matches), key=year_matches.count)

    # 라이선스 정보 추출
    license_info = None
    license_patterns = [
        r'\{\{(PD-[^}]+)\}\}',
        r'\{\{(CC-[^}]+)\}\}'
    ]
    for pattern in license_patterns:
        matches = re.findall(pattern, text)
        if matches:
            license_info = matches[0]
            break

    # 언어 정보 추출
    language = None
    if '한자' in text and '한글' in text:
        language = '한자+한글'
    elif '한자' in text:
        language = '한자'
    elif '한글' in text:
        language = '한글'

    # 중복 제거
    authors = list(set([a.strip() for a in authors if a.strip()]))
    categories = list(set([c.strip() for c in categories if c.strip()]))

    return {
        'authors': authors,
        'categories': categories,
        'composer': composer,
        'translator': None,  # 나중에 확장 가능
        'year': year,
        'license': license_info,
        'language': language
    }

def clean_text(text):
    """
    위키텍스트에서 마크업을 제거하고 깔끔한 본문만 추출합니다

    제거하는 것들:
    - 템플릿: {{...}}
    - HTML 태그: <div>, <br> 등
    - 위키 링크: [[...]]
    - 마크업: ''', ''
    """
    if not text:
        return ""

    # 템플릿 제거
    cleaned = re.sub(r'\{\{[^{}]*\}\}', '', text)

    # 링크에서 텍스트만 추출
    cleaned = re.sub(r'\[\[[^|\]]*\|([^\]]+)\]\]', r'\1', cleaned)
    cleaned = re.sub(r'\[\[([^\]]+)\]\]', r'\1', cleaned)

    # HTML 태그 제거
    cleaned = re.sub(r'<[^>]+>', '', cleaned)

    # 위키 마크업 제거
    cleaned = re.sub(r"'''([^']+)'''", r'\1', cleaned)  # 굵은 글씨
    cleaned = re.sub(r"''([^']+)''", r'\1', cleaned)   # 기울임

    # 섹션 헤더 제거
    cleaned = re.sub(r'^=+\s*([^=]+)\s*=+$', r'\1', cleaned, flags=re.MULTILINE)

    # 여러 공백을 하나로
    cleaned = re.sub(r'\s+', ' ', cleaned)

    return cleaned.strip()

def generate_urls(title, authors):
    """
    페이지와 저자의 URL을 생성합니다
    """
    base_url = "https://ko.wikisource.org/wiki/"

    # 페이지 URL
    page_url = base_url + urllib.parse.quote(title.replace(' ', '_'))

    # 저자 URL들
    author_links = []
    for author in authors:
        author_url = base_url + urllib.parse.quote(f"저자:{author}".replace(' ', '_'))
        author_links.append({
            'name': author,
            'url': author_url
        })

    return page_url, author_links

print("✅ 텍스트 파싱 함수들이 정의되었습니다!")

✅ 텍스트 파싱 함수들이 정의되었습니다!


## 🚀 API 보강 함수

덤프에서 파싱한 데이터를 API로 보강하는 핵심 함수입니다.

In [5]:
def enhance_with_api(page_data):
    """
    페이지 데이터를 API 정보로 보강합니다

    보강 과정:
    1. API에서 완전한 분류 정보 가져오기
    2. 위키데이터에서 연도 정보 가져오기
    3. 분류에서 연도 추출하기
    4. 최종 연도 결정 (분류 > 위키데이터 > 덤프)
    """
    title = page_data['title']

    # API에서 완전한 분류 정보 가져오기
    api_categories = get_categories_from_api(title)

    # 위키데이터에서 연도 정보 가져오기
    wikidata_year = get_year_from_wikidata(title)

    # 기존 데이터 복사
    enhanced = page_data.copy()

    # 분류 정보 병합 (중복 제거)
    all_categories = list(set(page_data.get('categories', []) + api_categories))
    enhanced['categories'] = all_categories
    enhanced['api_categories'] = api_categories

    # 분류에서 연도 추출
    year_from_categories = None
    year_categories = [cat for cat in all_categories if '년' in cat and '작품' in cat]
    if year_categories:
        for cat in year_categories:
            year_match = re.search(r'(\d{4})년', cat)
            if year_match:
                year_from_categories = int(year_match.group(1))
                break

    # 최종 연도 결정 (우선순위: 분류 > 위키데이터 > 덤프)
    enhanced['year'] = (
        year_from_categories or
        wikidata_year or
        page_data.get('year')
    )

    # 보강 정보 추가
    enhanced['year_from_categories'] = year_from_categories
    enhanced['year_from_wikidata'] = wikidata_year

    return enhanced

print("✅ API 보강 함수가 정의되었습니다!")

✅ API 보강 함수가 정의되었습니다!


## ⚡ 메인 파서 (멀티프로세싱)

모든 CPU 코어를 사용해서 빠르게 처리하는 메인 파서입니다.

In [6]:
def process_pages_batch(pages_batch):
    """
    페이지 배치를 처리하는 워커 함수
    (멀티프로세싱에서 각 코어가 실행)
    """
    results = []

    for page_data in pages_batch:
        try:
            page_id, title, namespace, redirect, revisions = page_data

            if not revisions:
                continue

            # 최신 리비전 사용
            revision = revisions[0]
            revision_id, timestamp, username, comment, text, size = revision

            # 메타데이터 추출
            metadata = extract_metadata(text)

            # 본문 정리
            clean_content = clean_text(text)

            # URL 생성
            page_url, author_links = generate_urls(title, metadata['authors'])

            # 페이지 데이터 구성
            page_result = {
                'page_id': page_id,
                'title': title,
                'url': page_url,
                'namespace': namespace,
                'redirect': redirect,

                # 필수 메타데이터
                'authors': metadata['authors'],
                'author_links': author_links,
                'categories': metadata['categories'],
                'content': clean_content,
                'raw_content': text,

                # 추가 메타데이터
                'composer': metadata['composer'],
                'translator': metadata['translator'],
                'year': metadata['year'],
                'license': metadata['license'],
                'language': metadata['language'],

                # 리비전 정보
                'revision_id': revision_id,
                'last_modified': str(timestamp) if timestamp else None,
                'last_contributor': username,
                'size': len(text) if text else 0,
                'content_size': len(clean_content)
            }

            results.append(page_result)

        except Exception as e:
            print(f"페이지 처리 오류: {e}")
            continue

    return results

def parse_wikisource(dump_file, limit=None, enable_api=False, batch_size=50):
    """
    위키문헌 덤프를 파싱합니다

    Args:
        dump_file: 덤프 파일 경로
        limit: 처리할 페이지 수 제한 (None이면 전체)
        enable_api: API 보강 사용 여부
        batch_size: 배치 크기

    Returns:
        list: 파싱된 페이지 데이터
    """
    start_time = time.time()

    # CPU 코어 수 (Colab에서 최대 활용)
    max_workers = mp.cpu_count()

    print(f"🚀 위키문헌 파싱 시작!")
    print(f"  📊 CPU 코어: {max_workers}개 (최대 활용)")
    print(f"  📦 배치 크기: {batch_size}")
    print(f"  🌐 API 보강: {'사용' if enable_api else '사용 안함'}")

    # 1단계: 덤프에서 데이터 수집
    print("\n📥 1단계: 덤프 데이터 수집")
    pages_batches = []
    current_batch = []
    total_pages = 0

    with bz2.open(dump_file, 'rt', encoding='utf-8') as f:
        dump = mwxml.Dump.from_file(f)

        for page in tqdm(dump, desc="페이지 수집"):
            if limit and total_pages >= limit:
                break

            try:
                # 리비전 데이터 수집
                revisions = []
                for revision in page:
                    revisions.append((
                        revision.id,
                        revision.timestamp,
                        revision.user.text if revision.user else None,
                        revision.comment,
                        revision.text,
                        revision.bytes
                    ))
                    break  # 최신 리비전만

                page_data = (
                    page.id,
                    page.title,
                    page.namespace,
                    str(page.redirect.title) if page.redirect else None,
                    revisions
                )
                current_batch.append(page_data)

                if len(current_batch) >= batch_size:
                    pages_batches.append(current_batch)
                    current_batch = []

                total_pages += 1

            except Exception as e:
                continue

        if current_batch:
            pages_batches.append(current_batch)

    print(f"✅ {total_pages}개 페이지를 {len(pages_batches)}개 배치로 수집")

    # 2단계: 멀티프로세싱으로 병렬 처리
    print("\n⚡ 2단계: 멀티프로세싱 처리")
    results = []

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # 모든 배치를 병렬로 처리
        futures = [executor.submit(process_pages_batch, batch) for batch in pages_batches]

        for future in tqdm(as_completed(futures), total=len(futures), desc="배치 처리"):
            try:
                batch_results = future.result()
                results.extend(batch_results)
            except Exception as e:
                print(f"배치 처리 오류: {e}")

    # 3단계: API 보강 (선택적)
    if enable_api and results:
        print("\n🌐 3단계: API 보강 처리")
        enhanced_results = []

        for page_data in tqdm(results, desc="API 보강"):
            try:
                enhanced = enhance_with_api(page_data)
                enhanced_results.append(enhanced)
                time.sleep(0.1)  # API 제한 고려
            except Exception as e:
                print(f"API 보강 오류 ({page_data.get('title', 'Unknown')}): {e}")
                enhanced_results.append(page_data)

        results = enhanced_results

    total_time = time.time() - start_time
    print(f"\n🎉 파싱 완료!")
    print(f"  ⏱️  총 시간: {total_time:.1f}초")
    print(f"  📊 처리 속도: {len(results)/total_time:.1f} 페이지/초")
    print(f"  📄 총 페이지: {len(results)}개")

    return results

print("✅ 메인 파서가 정의되었습니다!")

✅ 메인 파서가 정의되었습니다!


## 🎯 실습 1: 애국가 분석 (기본 파싱)

먼저 XML 덤프만으로 애국가를 파싱해보겠습니다.

In [7]:
# 애국가만 파싱 (API 보강 없음)
print("📚 실습 1: 기본 XML 덤프 파싱")
print("="*50)

basic_results = parse_wikisource(
    dump_file,
    limit=1,  # 첫 번째 페이지 (애국가)만
    enable_api=False,  # API 보강 안함
    batch_size=1
)

if basic_results:
    aegukga_basic = basic_results[0]

    print(f"\n📄 제목: {aegukga_basic['title']}")
    print(f"👤 저자: {aegukga_basic['authors']}")
    print(f"📂 분류: {aegukga_basic['categories']}")
    print(f"🎵 작곡가: {aegukga_basic.get('composer', 'N/A')}")
    print(f"📅 연도: {aegukga_basic.get('year', 'N/A')}")
    print(f"📜 라이선스: {aegukga_basic.get('license', 'N/A')}")
    print(f"📏 본문 길이: {aegukga_basic['content_size']} 문자")

    print(f"\n📖 본문 미리보기:")
    print(f"{aegukga_basic['content'][:200]}...")

    print(f"\n🔍 문제점 발견:")
    print(f"  ❌ 분류가 {len(aegukga_basic['categories'])}개만 나옴 (실제로는 4개)")
    print(f"  ❌ '1941년 작품' 분류가 누락됨")
    print(f"  ❌ 'PD-old-50' 분류가 누락됨")
else:
    print("❌ 데이터를 찾을 수 없습니다.")

📚 실습 1: 기본 XML 덤프 파싱
🚀 위키문헌 파싱 시작!
  📊 CPU 코어: 2개 (최대 활용)
  📦 배치 크기: 1
  🌐 API 보강: 사용 안함

📥 1단계: 덤프 데이터 수집


페이지 수집: 1it [00:00, 304.91it/s]

✅ 1개 페이지를 1개 배치로 수집

⚡ 2단계: 멀티프로세싱 처리



배치 처리: 100%|██████████| 1/1 [00:00<00:00, 60.14it/s]


🎉 파싱 완료!
  ⏱️  총 시간: 0.1초
  📊 처리 속도: 11.0 페이지/초
  📄 총 페이지: 1개

📄 제목: 애국가 (대한민국)
👤 저자: ['윤치호']
📂 분류: ['대한민국의 노래', '국가']
🎵 작곡가: 안익태(安益泰)
📅 연도: None
📜 라이선스: PD-old-50
📏 본문 길이: 426 문자

📖 본문 미리보기:
한자 혼용 ;1 :東海물과 白頭山이 마르고 닳도록하느님이 保佑하사 우리 나라 萬歲 :無窮花 三千里 華麗江山大韓사람 大韓으로 길이 保全하세 ;2 :南山 위에 저 소나무 鐵甲을 두른 듯바람 서리 不變함은 우리 氣像일세 ;3 :가을 하늘 空豁한데 높고 구름 없이밝은 달은 우리 가슴 一片丹心일세 ;4 :이 氣像과 이 맘으로 忠誠을 다하여괴로우나 즐거우나 나라 사랑...

🔍 문제점 발견:
  ❌ 분류가 2개만 나옴 (실제로는 4개)
  ❌ '1941년 작품' 분류가 누락됨
  ❌ 'PD-old-50' 분류가 누락됨


## 🌟 실습 2: 애국가 분석 (API 보강)

이제 API 보강을 사용해서 완전한 정보를 추출해보겠습니다.

In [8]:
# 애국가 API 보강 파싱
print("🌟 실습 2: API 보강 파싱")
print("="*50)

api_results = parse_wikisource(
    dump_file,
    limit=1,  # 첫 번째 페이지 (애국가)만
    enable_api=True,  # API 보강 사용!
    batch_size=1
)

if api_results:
    aegukga_api = api_results[0]

    print(f"\n📄 제목: {aegukga_api['title']}")
    print(f"👤 저자: {aegukga_api['authors']}")

    # 분류 비교
    dump_cats = [cat for cat in aegukga_api['categories'] if cat not in aegukga_api.get('api_categories', [])]
    api_cats = aegukga_api.get('api_categories', [])

    print(f"\n📂 분류 정보:")
    print(f"  덤프에서: {dump_cats}")
    print(f"  API에서: {api_cats}")
    print(f"  전체: {aegukga_api['categories']}")

    print(f"\n🎵 작곡가: {aegukga_api.get('composer', 'N/A')}")

    # 연도 정보 상세
    print(f"\n📅 연도 정보:")
    print(f"  최종: {aegukga_api.get('year', 'N/A')}")
    print(f"  분류에서: {aegukga_api.get('year_from_categories', 'N/A')}")
    print(f"  위키데이터에서: {aegukga_api.get('year_from_wikidata', 'N/A')}")

    print(f"\n📜 라이선스: {aegukga_api.get('license', 'N/A')}")
    print(f"🌐 언어: {aegukga_api.get('language', 'N/A')}")
    print(f"📏 본문 길이: {aegukga_api['content_size']} 문자")

    # 성공 확인
    success_checks = [
        ('✅' if '1941년 작품' in api_cats else '❌', '1941년 작품 분류'),
        ('✅' if 'PD-old-50' in api_cats else '❌', 'PD-old-50 분류'),
        ('✅' if aegukga_api.get('year') == 1941 else '❌', '1941년 연도 추출'),
        ('✅' if len(api_cats) >= 4 else '❌', '4개 분류 완전 추출')
    ]

    print(f"\n🎯 목표 달성:")
    for status, description in success_checks:
        print(f"  {status} {description}")

else:
    print("❌ 데이터를 찾을 수 없습니다.")

🌟 실습 2: API 보강 파싱
🚀 위키문헌 파싱 시작!
  📊 CPU 코어: 2개 (최대 활용)
  📦 배치 크기: 1
  🌐 API 보강: 사용

📥 1단계: 덤프 데이터 수집


페이지 수집: 1it [00:00, 712.35it/s]

✅ 1개 페이지를 1개 배치로 수집

⚡ 2단계: 멀티프로세싱 처리



배치 처리: 100%|██████████| 1/1 [00:00<00:00, 37.76it/s]



🌐 3단계: API 보강 처리


API 보강: 100%|██████████| 1/1 [00:00<00:00,  1.35it/s]


🎉 파싱 완료!
  ⏱️  총 시간: 0.9초
  📊 처리 속도: 1.2 페이지/초
  📄 총 페이지: 1개

📄 제목: 애국가 (대한민국)
👤 저자: ['윤치호']

📂 분류 정보:
  덤프에서: []
  API에서: ['1941년 작품', 'PD-old-50', '국가', '대한민국의 노래']
  전체: ['PD-old-50', '1941년 작품', '대한민국의 노래', '국가']

🎵 작곡가: 안익태(安益泰)

📅 연도 정보:
  최종: 1941
  분류에서: 1941
  위키데이터에서: 1941

📜 라이선스: PD-old-50
🌐 언어: 한자+한글
📏 본문 길이: 426 문자

🎯 목표 달성:
  ✅ 1941년 작품 분류
  ✅ PD-old-50 분류
  ✅ 1941년 연도 추출
  ✅ 4개 분류 완전 추출


## 📊 비교 분석

기본 파싱과 API 보강 파싱의 차이점을 비교해보겠습니다.

In [9]:
print("📊 기본 파싱 vs API 보강 파싱 비교")
print("="*60)

if basic_results and api_results:
    basic = basic_results[0]
    enhanced = api_results[0]

    print(f"\n📂 분류 정보:")
    print(f"  기본 파싱: {len(basic['categories'])}개 → {basic['categories']}")
    print(f"  API 보강: {len(enhanced['categories'])}개 → {enhanced['categories']}")
    print(f"  개선: +{len(enhanced['categories']) - len(basic['categories'])}개 분류 추가")

    print(f"\n📅 연도 정보:")
    print(f"  기본 파싱: {basic.get('year', 'None')}")
    print(f"  API 보강: {enhanced.get('year', 'None')} (분류: {enhanced.get('year_from_categories')}, 위키데이터: {enhanced.get('year_from_wikidata')})")

    print(f"\n🎯 핵심 성과:")
    missing_categories = set(enhanced['categories']) - set(basic['categories'])
    if missing_categories:
        print(f"  ✅ 추가된 분류: {list(missing_categories)}")

    if enhanced.get('year') and not basic.get('year'):
        print(f"  ✅ 연도 정보 추출 성공: {enhanced.get('year')}년")

    if enhanced.get('year_from_categories') == enhanced.get('year_from_wikidata'):
        print(f"  ✅ 분류와 위키데이터 연도 일치: {enhanced.get('year')}년")

else:
    print("❌ 비교할 데이터가 없습니다.")

print(f"\n💡 학습 포인트:")
print(f"  1. XML 덤프만으로는 템플릿 기반 분류를 놓칠 수 있음")
print(f"  2. API 보강으로 웹사이트와 동일한 완전한 정보 추출 가능")
print(f"  3. 여러 소스에서 연도 정보를 교차 검증하여 정확성 향상")
print(f"  4. 구조화된 메타데이터로 후속 분석 작업 용이")

📊 기본 파싱 vs API 보강 파싱 비교

📂 분류 정보:
  기본 파싱: 2개 → ['대한민국의 노래', '국가']
  API 보강: 4개 → ['PD-old-50', '1941년 작품', '대한민국의 노래', '국가']
  개선: +2개 분류 추가

📅 연도 정보:
  기본 파싱: None
  API 보강: 1941 (분류: 1941, 위키데이터: 1941)

🎯 핵심 성과:
  ✅ 추가된 분류: ['PD-old-50', '1941년 작품']
  ✅ 연도 정보 추출 성공: 1941년
  ✅ 분류와 위키데이터 연도 일치: 1941년

💡 학습 포인트:
  1. XML 덤프만으로는 템플릿 기반 분류를 놓칠 수 있음
  2. API 보강으로 웹사이트와 동일한 완전한 정보 추출 가능
  3. 여러 소스에서 연도 정보를 교차 검증하여 정확성 향상
  4. 구조화된 메타데이터로 후속 분석 작업 용이


## 🔄 실습 3: 대량 처리 (선택적)

더 많은 페이지를 처리해보고 싶다면 아래 셀을 실행하세요.

In [10]:
# 🚀 대량 처리 실행
print("🚀 실습 3: 대량 처리")
print("="*50)

# 처리할 페이지 수 설정 (필요에 따라 조정)
BULK_LIMIT = 50  # 50개 페이지 처리
API_ENHANCEMENT = True  # API 보강 사용 여부

print(f"📊 설정:")
print(f"  처리 페이지: {BULK_LIMIT}개")
print(f"  API 보강: {'사용' if API_ENHANCEMENT else '사용 안함'}")
print(f"  CPU 활용: {mp.cpu_count()}개 코어 최대 활용")

# 대량 처리 실행
bulk_results = parse_wikisource(
    dump_file,
    limit=BULK_LIMIT,
    enable_api=API_ENHANCEMENT,
    batch_size=10  # 배치 크기
)

print(f"\n✅ 대량 처리 완료!")
print(f"처리된 페이지: {len(bulk_results)}개")

🚀 실습 3: 대량 처리
📊 설정:
  처리 페이지: 50개
  API 보강: 사용
  CPU 활용: 2개 코어 최대 활용
🚀 위키문헌 파싱 시작!
  📊 CPU 코어: 2개 (최대 활용)
  📦 배치 크기: 10
  🌐 API 보강: 사용

📥 1단계: 덤프 데이터 수집


페이지 수집: 50it [00:00, 119.69it/s]

✅ 50개 페이지를 5개 배치로 수집

⚡ 2단계: 멀티프로세싱 처리



배치 처리: 100%|██████████| 5/5 [00:11<00:00,  2.32s/it]



🌐 3단계: API 보강 처리


API 보강: 100%|██████████| 50/50 [00:25<00:00,  1.93it/s]


🎉 파싱 완료!
  ⏱️  총 시간: 38.0초
  📊 처리 속도: 1.3 페이지/초
  📄 총 페이지: 50개

✅ 대량 처리 완료!
처리된 페이지: 50개


## 📊 대량 처리 결과 분석

처리된 데이터의 통계와 샘플을 확인해보겠습니다.

In [11]:
if 'bulk_results' in locals() and bulk_results:
    print("📊 대량 처리 결과 분석")
    print("="*50)

    # 기본 통계
    total_pages = len(bulk_results)
    api_enhanced = sum(1 for page in bulk_results if page.get('api_categories'))
    year_found = sum(1 for page in bulk_results if page.get('year'))
    authors_found = sum(1 for page in bulk_results if page.get('authors'))
    content_pages = sum(1 for page in bulk_results if page.get('content_size', 0) > 100)

    print(f"📈 처리 통계:")
    print(f"  총 페이지: {total_pages}개")
    print(f"  API 보강된 페이지: {api_enhanced}개 ({api_enhanced/total_pages*100:.1f}%)")
    print(f"  연도 정보 있는 페이지: {year_found}개 ({year_found/total_pages*100:.1f}%)")
    print(f"  저자 정보 있는 페이지: {authors_found}개 ({authors_found/total_pages*100:.1f}%)")
    print(f"  실질적 본문 있는 페이지: {content_pages}개 ({content_pages/total_pages*100:.1f}%)")

    # 분류 통계
    all_categories = []
    for page in bulk_results:
        all_categories.extend(page.get('categories', []))

    category_counts = {}
    for cat in all_categories:
        category_counts[cat] = category_counts.get(cat, 0) + 1

    top_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    print(f"\n📂 상위 10개 분류:")
    for cat, count in top_categories:
        print(f"  {cat}: {count}개")

    # 연도별 통계
    years = [page.get('year') for page in bulk_results if page.get('year')]
    if years:
        year_counts = {}
        for year in years:
            year_counts[year] = year_counts.get(year, 0) + 1

        print(f"\n📅 연도별 분포:")
        sorted_years = sorted(year_counts.items(), key=lambda x: int(x[0]) if str(x[0]).isdigit() else 0)
        for year, count in sorted_years:
            print(f"  {year}년: {count}개")

    # 샘플 페이지 표시
    print(f"\n📝 샘플 페이지 (상위 10개):")
    for i, page in enumerate(bulk_results[:10], 1):
        title = page['title']
        categories_count = len(page.get('categories', []))
        year = page.get('year', 'N/A')
        content_size = page.get('content_size', 0)

        print(f"  {i:2d}. {title}")
        print(f"      분류: {categories_count}개, 연도: {year}, 본문: {content_size}자")

else:
    print("❌ 대량 처리 결과가 없습니다. 먼저 위의 대량 처리를 실행하세요.")


📊 대량 처리 결과 분석
📈 처리 통계:
  총 페이지: 50개
  API 보강된 페이지: 28개 (56.0%)
  연도 정보 있는 페이지: 26개 (52.0%)
  저자 정보 있는 페이지: 8개 (16.0%)
  실질적 본문 있는 페이지: 42개 (84.0%)

📂 상위 10개 분류:
  PD-South Korea-exempt: 11개
  연도 미입력 작품: 6개
  도움말: 4개
  동음이의어 문서: 4개
  PD-old-100: 3개
  대한민국의 일부개정된 법률: 3개
  대한민국의 개정된 법률: 3개
  PD-old-70: 3개
  한글: 2개
  대한민국의 타법개정된 법률: 2개

📅 연도별 분포:
  1429년: 1개
  1446년: 1개
  1446년: 1개
  1459년: 1개
  1541년: 1개
  1926년: 1개
  1933년: 1개
  1934년: 1개
  1936년: 1개
  1936년: 1개
  1941년: 1개
  1943년: 1개
  1948년: 1개
  1955년: 1개
  1956년: 1개
  1959년: 1개
  1962년: 1개
  1965년: 2개
  1987년: 1개
  1998년: 1개
  2003년: 1개
  2004년: 1개
  2010년: 1개
  2014년: 1개
  2019년: 1개

📝 샘플 페이지 (상위 10개):
   1. 기독교 강요
      분류: 1개, 연도: 1541, 본문: 2782자
   2. 훈민정음
      분류: 3개, 연도: 1446, 본문: 5551자
   3. 대문
      분류: 0개, 연도: 1955, 본문: 909자
   4. 자매프로젝트
      분류: 0개, 연도: None, 본문: 1511자
   5. 위키미디어 재단
      분류: 1개, 연도: 2003, 본문: 1175자
   6. 도움말
      분류: 1개, 연도: None, 본문: 1220자
   7. FAQ
      분류: 1개, 연도: None, 본문: 4023자
   8. 저작권법 (대한민국,

## 💾 대량 처리 결과 저장 (CSV + JSON)

처리된 데이터를 다양한 형식으로 저장하여 후속 분석에 활용할 수 있도록 합니다.

In [12]:
def save_bulk_results(data, base_filename="kowikisource_bulk"):
    """
    대량 처리 결과를 다양한 형식으로 저장

    저장 형식:
    1. 완전한 JSON (모든 메타데이터 포함)
    2. 요약 CSV (핵심 정보만)
    3. 상세 CSV (API 보강 정보 포함)
    """
    if not data:
        print("❌ 저장할 데이터가 없습니다.")
        return

    timestamp = time.strftime("%Y%m%d_%H%M%S")

    print(f"💾 대량 처리 결과 저장 중...")
    print(f"📊 데이터: {len(data)}개 페이지")

    # 1. 완전한 JSON 저장 (모든 데이터)
    json_filename = f"{base_filename}_{timestamp}_complete.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    file_size_mb = os.path.getsize(json_filename) / (1024 * 1024)
    print(f"  ✅ 완전한 JSON: {json_filename} ({file_size_mb:.1f}MB)")

    # 2. 요약 CSV 저장 (핵심 정보만)
    summary_csv_data = []
    for page in data:
        summary_csv_data.append({
            'page_id': page['page_id'],
            'title': page['title'],
            'url': page['url'],
            'namespace': page['namespace'],
            'authors': ', '.join(page['authors']) if page['authors'] else '',
            'categories': ', '.join(page['categories']) if page['categories'] else '',
            'composer': page.get('composer', ''),
            'translator': page.get('translator', ''),
            'year': page.get('year', ''),
            'license': page.get('license', ''),
            'language': page.get('language', ''),
            'content_length': page.get('content_size', 0),
            'last_modified': page.get('last_modified', ''),
            'last_contributor': page.get('last_contributor', '')
        })

    summary_csv_filename = f"{base_filename}_{timestamp}_summary.csv"
    df_summary = pd.DataFrame(summary_csv_data)
    df_summary.to_csv(summary_csv_filename, index=False, encoding='utf-8')
    print(f"  ✅ 요약 CSV: {summary_csv_filename}")

    # 3. 상세 CSV 저장 (API 보강 정보 포함)
    detailed_csv_data = []
    for page in data:
        # API 보강 정보 분리
        dump_categories = [cat for cat in page.get('categories', []) if cat not in page.get('api_categories', [])]
        api_categories = page.get('api_categories', [])

        detailed_csv_data.append({
            'page_id': page['page_id'],
            'title': page['title'],
            'url': page['url'],
            'namespace': page['namespace'],
            'authors': ', '.join(page['authors']) if page['authors'] else '',
            'dump_categories': ', '.join(dump_categories),
            'api_categories': ', '.join(api_categories),
            'all_categories': ', '.join(page['categories']) if page['categories'] else '',
            'composer': page.get('composer', ''),
            'year_final': page.get('year', ''),
            'year_from_categories': page.get('year_from_categories', ''),
            'year_from_wikidata': page.get('year_from_wikidata', ''),
            'license': page.get('license', ''),
            'language': page.get('language', ''),
            'content_length': page.get('content_size', 0),
            'raw_content_length': page.get('size', 0),
            'last_modified': page.get('last_modified', ''),
            'last_contributor': page.get('last_contributor', '')
        })

    detailed_csv_filename = f"{base_filename}_{timestamp}_detailed.csv"
    df_detailed = pd.DataFrame(detailed_csv_data)
    df_detailed.to_csv(detailed_csv_filename, index=False, encoding='utf-8')
    print(f"  ✅ 상세 CSV: {detailed_csv_filename}")

    # 4. 메타데이터 요약 저장
    metadata = {
        'generation_info': {
            'timestamp': timestamp,
            'total_pages': len(data),
            'api_enhanced_pages': sum(1 for page in data if page.get('api_categories')),
            'pages_with_year': sum(1 for page in data if page.get('year')),
            'pages_with_authors': sum(1 for page in data if page.get('authors')),
            'pages_with_content': sum(1 for page in data if page.get('content_size', 0) > 100)
        },
        'files_generated': {
            'complete_json': json_filename,
            'summary_csv': summary_csv_filename,
            'detailed_csv': detailed_csv_filename
        }
    }

    metadata_filename = f"{base_filename}_{timestamp}_metadata.json"
    with open(metadata_filename, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)

    print(f"  ✅ 메타데이터: {metadata_filename}")

    print(f"\n📁 생성된 파일들:")
    print(f"  📄 {json_filename} - 완전한 데이터 (JSON)")
    print(f"  📊 {summary_csv_filename} - 핵심 정보 (CSV)")
    print(f"  📈 {detailed_csv_filename} - API 보강 정보 포함 (CSV)")
    print(f"  ℹ️  {metadata_filename} - 생성 정보 (JSON)")

    return {
        'json_file': json_filename,
        'summary_csv': summary_csv_filename,
        'detailed_csv': detailed_csv_filename,
        'metadata_file': metadata_filename
    }

print("✅ 대량 저장 함수가 정의되었습니다!")

✅ 대량 저장 함수가 정의되었습니다!


In [13]:
# 대량 처리 결과 저장 실행
if 'bulk_results' in locals() and bulk_results:
    print("💾 대량 처리 결과를 여러 형식으로 저장합니다...")
    saved_files = save_bulk_results(bulk_results)

    print(f"\n🎯 파일 활용 가이드:")
    print(f"  📊 pandas 분석:")
    print(f"     df = pd.read_csv('{saved_files['summary_csv']}')")
    print(f"     df.head()")

    print(f"\n  🔍 상세 분석:")
    print(f"     detailed_df = pd.read_csv('{saved_files['detailed_csv']}')")
    print(f"     # API 보강 효과 확인")
    print(f"     detailed_df[['dump_categories', 'api_categories']].head()")

    print(f"\n  📄 완전한 데이터:")
    print(f"     with open('{saved_files['json_file']}', 'r') as f:")
    print(f"         data = json.load(f)")

    # 간단한 데이터 검증
    print(f"\n✅ 저장 완료 확인:")

    # CSV 파일 검증
    try:
        test_df = pd.read_csv(saved_files['summary_csv'])
        print(f"  📊 CSV 파일: {len(test_df)}행 × {len(test_df.columns)}열")
        print(f"     주요 컬럼: {', '.join(test_df.columns[:5])}...")
    except Exception as e:
        print(f"  ❌ CSV 검증 실패: {e}")

    # JSON 파일 검증
    try:
        with open(saved_files['json_file'], 'r', encoding='utf-8') as f:
            test_json = json.load(f)
        print(f"  📄 JSON 파일: {len(test_json)}개 항목")
        if test_json:
            print(f"     키 개수: {len(test_json[0].keys())}개")
    except Exception as e:
        print(f"  ❌ JSON 검증 실패: {e}")

else:
    print("❌ 저장할 대량 처리 결과가 없습니다.")
    print("💡 위의 '대량 처리 실행' 셀을 먼저 실행하세요.")

💾 대량 처리 결과를 여러 형식으로 저장합니다...
💾 대량 처리 결과 저장 중...
📊 데이터: 50개 페이지
  ✅ 완전한 JSON: kowikisource_bulk_20250911_024137_complete.json (3.3MB)
  ✅ 요약 CSV: kowikisource_bulk_20250911_024137_summary.csv
  ✅ 상세 CSV: kowikisource_bulk_20250911_024137_detailed.csv
  ✅ 메타데이터: kowikisource_bulk_20250911_024137_metadata.json

📁 생성된 파일들:
  📄 kowikisource_bulk_20250911_024137_complete.json - 완전한 데이터 (JSON)
  📊 kowikisource_bulk_20250911_024137_summary.csv - 핵심 정보 (CSV)
  📈 kowikisource_bulk_20250911_024137_detailed.csv - API 보강 정보 포함 (CSV)
  ℹ️  kowikisource_bulk_20250911_024137_metadata.json - 생성 정보 (JSON)

🎯 파일 활용 가이드:
  📊 pandas 분석:
     df = pd.read_csv('kowikisource_bulk_20250911_024137_summary.csv')
     df.head()

  🔍 상세 분석:
     detailed_df = pd.read_csv('kowikisource_bulk_20250911_024137_detailed.csv')
     # API 보강 효과 확인
     detailed_df[['dump_categories', 'api_categories']].head()

  📄 완전한 데이터:
     with open('kowikisource_bulk_20250911_024137_complete.json', 'r') as f:
         data = json.lo

## 🔍 애국가 가사 검증

사용자 요청에 따라 애국가 본문(가사)이 제대로 파싱되었는지 확인해보겠습니다.

In [14]:
# 애국가 본문 검증
print("🔍 애국가 가사 검증")
print("="*50)

# API 보강 결과에서 애국가 확인
if 'api_results' in locals() and api_results:
    aegukga = api_results[0]

    print(f"📄 페이지: {aegukga['title']}")
    print(f"🌐 URL: {aegukga['url']}")
    print(f"📏 전체 본문 길이: {aegukga['content_size']} 문자")
    print(f"📏 원본 길이: {aegukga.get('size', 0)} 문자")

    # 본문 내용 확인
    content = aegukga['content']
    raw_content = aegukga.get('raw_content', '')

    print(f"\n📖 정제된 본문:")
    print(f"{content}")

    # 가사 키워드 확인
    lyrics_keywords = ['동해물과', '백두산이', '하느님이', '보우하사', '무궁화']
    found_keywords = []

    for keyword in lyrics_keywords:
        if keyword in content:
            found_keywords.append(keyword)

    print(f"\n🎵 가사 키워드 확인:")
    for keyword in lyrics_keywords:
        status = '✅' if keyword in content else '❌'
        print(f"  {status} '{keyword}' {'발견됨' if keyword in content else '없음'}")

    # 한자/한글 분석
    import re

    # 한자 패턴 확인
    hanja_pattern = r'[\u4e00-\u9fff]+'
    hangul_pattern = r'[\uac00-\ud7af]+'

    hanja_matches = re.findall(hanja_pattern, content)
    hangul_matches = re.findall(hangul_pattern, content)

    print(f"\n🔤 언어 분석:")
    print(f"  ✅ 한자 구문 발견: {len(hanja_matches)}개")
    if hanja_matches:
        print(f"     예시: {', '.join(hanja_matches[:3])}...")

    print(f"  ✅ 한글 구문 발견: {len(hangul_matches)}개")
    if hangul_matches:
        print(f"     예시: {', '.join(hangul_matches[:3])}...")

    # 가사 구조 확인
    lines = [line.strip() for line in content.split('\n') if line.strip()]
    meaningful_lines = [line for line in lines if len(line) > 3]

    print(f"\n📝 가사 구조:")
    print(f"  전체 줄 수: {len(lines)}줄")
    print(f"  의미있는 줄: {len(meaningful_lines)}줄")

    print(f"\n🎼 가사 내용 (처음 10줄):")
    for i, line in enumerate(meaningful_lines[:10], 1):
        print(f"  {i:2d}. {line}")

    # 성공 여부 판단
    success_criteria = [
        (len(found_keywords) >= 3, f"주요 가사 키워드 {len(found_keywords)}/5개 발견"),
        (len(hanja_matches) > 0, "한자 가사 보존됨"),
        (len(hangul_matches) > 0, "한글 가사 보존됨"),
        (aegukga['content_size'] > 300, f"충분한 본문 길이 ({aegukga['content_size']}자)"),
        (len(meaningful_lines) >= 5, f"가사 구조 보존 ({len(meaningful_lines)}줄)")
    ]

    print(f"\n🎯 파싱 성공 여부:")
    all_success = True
    for success, description in success_criteria:
        status = '✅' if success else '❌'
        print(f"  {status} {description}")
        if not success:
            all_success = False

    if all_success:
        print(f"\n🎉 애국가 가사 파싱 완벽 성공!")
        print(f"✅ 한자 가사 보존됨")
        print(f"✅ 한글 가사 보존됨")
        print(f"✅ 가사 구조 완전 보존됨")
    else:
        print(f"\n⚠️  일부 문제가 발견되었습니다.")

else:
    print("❌ 애국가 데이터를 찾을 수 없습니다.")
    print("💡 먼저 'API 보강 파싱' 셀을 실행하세요.")

🔍 애국가 가사 검증
📄 페이지: 애국가 (대한민국)
🌐 URL: https://ko.wikisource.org/wiki/%EC%95%A0%EA%B5%AD%EA%B0%80_%28%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%29
📏 전체 본문 길이: 426 문자
📏 원본 길이: 866 문자

📖 정제된 본문:
한자 혼용 ;1 :東海물과 白頭山이 마르고 닳도록하느님이 保佑하사 우리 나라 萬歲 :無窮花 三千里 華麗江山大韓사람 大韓으로 길이 保全하세 ;2 :南山 위에 저 소나무 鐵甲을 두른 듯바람 서리 不變함은 우리 氣像일세 ;3 :가을 하늘 空豁한데 높고 구름 없이밝은 달은 우리 가슴 一片丹心일세 ;4 :이 氣像과 이 맘으로 忠誠을 다하여괴로우나 즐거우나 나라 사랑하세 한글 ;1 :동해 물과 백두산이 마르고 닳도록하느님이 보우하사 우리나라 만세 :무궁화 삼천리 화려강산대한 사람 대한으로 길이 보전하세 ;2 :남산 위에 저 소나무 철갑을 두른 듯바람 서리 불변함은 우리 기상일세 ;3 :가을 하늘 공활한데 높고 구름 없이밝은 달은 우리 가슴 일편단심일세 ;4 :이 기상과 이 맘으로 충성을 다하여괴로우나 즐거우나 나라 사랑하세 라이선스 분류:국가 분류:대한민국의 노래

🎵 가사 키워드 확인:
  ❌ '동해물과' 없음
  ✅ '백두산이' 발견됨
  ✅ '하느님이' 발견됨
  ✅ '보우하사' 발견됨
  ✅ '무궁화' 발견됨

🔤 언어 분석:
  ✅ 한자 구문 발견: 17개
     예시: 東海, 白頭山, 保佑...
  ✅ 한글 구문 발견: 94개
     예시: 한자, 혼용, 물과...

📝 가사 구조:
  전체 줄 수: 1줄
  의미있는 줄: 1줄

🎼 가사 내용 (처음 10줄):
   1. 한자 혼용 ;1 :東海물과 白頭山이 마르고 닳도록하느님이 保佑하사 우리 나라 萬歲 :無窮花 三千里 華麗江山大韓사람 大韓으로 길이 保全하세 ;2 :南山 위에 저 소나무 鐵甲을 두른 듯바람 서리 不變함은 우리 氣像일세 ;3 :가을 하늘 空豁한데 높고 구름 없이밝은 달

## 🚀 전체 위키문헌 파싱 (고급)

**주의**: 이 섹션은 전체 한국어 위키문헌을 파싱합니다. 수천 개의 페이지를 처리하므로 시간이 오래 걸릴 수 있습니다.

### ⚠️ 실행 전 확인사항:
- Colab Pro 사용 권장 (더 많은 메모리와 시간 제한)
- 안정적인 인터넷 연결 필요 (API 호출)
- 완료까지 30분-2시간 소요 예상

In [18]:
# 전체 위키문헌 파싱 설정
print("🚀 전체 위키문헌 파싱 준비")
print("="*50)

# 파싱 설정
FULL_PARSING_CONFIG = {
    'enable_api': True,           # API 보강 사용 (권장)
    'batch_size': 20,            # 배치 크기 (메모리 고려)
    'api_delay': 0.05,           # API 호출 간격 (초)
    'save_interval': 1000,       # 중간 저장 간격
    'max_retries': 3,            # API 재시도 횟수
}

print(f"📊 설정:")
print(f"  API 보강: {'✅' if FULL_PARSING_CONFIG['enable_api'] else '❌'}")
print(f"  배치 크기: {FULL_PARSING_CONFIG['batch_size']}")
print(f"  CPU 활용: {mp.cpu_count()}개 코어")
print(f"  중간 저장: {FULL_PARSING_CONFIG['save_interval']}개마다")

# 예상 시간 계산
import os
dump_size = os.path.getsize(dump_file) / (1024 * 1024)  # MB
estimated_pages = int(dump_size * 10)  # 대략적 추정

print(f"\n📈 예상 정보:")
print(f"  덤프 크기: {dump_size:.1f}MB")
print(f"  예상 페이지: ~{estimated_pages:,}개")
print(f"  예상 시간: {estimated_pages/100:.0f}-{estimated_pages/50:.0f}분 (API 포함)")

# 사용자 확인
print(f"\n⚠️  전체 파싱을 실행하려면:")
print(f"  1. 아래 EXECUTE_FULL_PARSING을 True로 변경")
print(f"  2. 셀을 다시 실행")
print(f"  3. 진행상황을 모니터링")

# 안전장치
EXECUTE_FULL_PARSING = True  # 전체 파싱 실행하려면 True로 변경

if EXECUTE_FULL_PARSING:
    print(f"\n🔥 전체 파싱을 시작합니다...")
else:
    print(f"\n💡 아직 실행되지 않습니다. EXECUTE_FULL_PARSING = True로 설정하세요.")

🚀 전체 위키문헌 파싱 준비
📊 설정:
  API 보강: ✅
  배치 크기: 20
  CPU 활용: 2개 코어
  중간 저장: 1000개마다

📈 예상 정보:
  덤프 크기: 147.8MB
  예상 페이지: ~1,477개
  예상 시간: 15-30분 (API 포함)

⚠️  전체 파싱을 실행하려면:
  1. 아래 EXECUTE_FULL_PARSING을 True로 변경
  2. 셀을 다시 실행
  3. 진행상황을 모니터링

🔥 전체 파싱을 시작합니다...


In [ ]:
# 향상된 전체 파싱 함수 (중간 저장 포함)
def parse_full_wikisource(dump_file, config):
    """
    전체 위키문헌을 파싱하고 중간에 저장하는 함수
    """
    start_time = time.time()

    print(f"🚀 전체 위키문헌 파싱 시작!")
    print(f"  📊 CPU 코어: {mp.cpu_count()}개 활용")
    print(f"  🌐 API 보강: {'사용' if config['enable_api'] else '사용 안함'}")
    print(f"  💾 중간 저장: {config['save_interval']}개마다")

    # 1단계: 모든 페이지 수집
    print(f"\n📥 1단계: 전체 덤프 데이터 수집")
    all_pages = []
    total_pages = 0

    with bz2.open(dump_file, 'rt', encoding='utf-8') as f:
        dump = mwxml.Dump.from_file(f)

        for page in tqdm(dump, desc="전체 페이지 수집"):
            try:
                # 리비전 데이터 수집
                revisions = []
                for revision in page:
                    revisions.append((
                        revision.id,
                        revision.timestamp,
                        revision.user.text if revision.user else None,
                        revision.comment,
                        revision.text,
                        revision.bytes
                    ))
                    break

                if revisions:  # 리비전이 있는 페이지만
                    page_data = (
                        page.id,
                        page.title,
                        page.namespace,
                        str(page.redirect.title) if page.redirect else None,
                        revisions
                    )
                    all_pages.append(page_data)
                    total_pages += 1

            except Exception as e:
                continue

    print(f"✅ 총 {total_pages:,}개 페이지 수집 완료")

    # 2단계: 배치로 나누기
    batch_size = config['batch_size']
    batches = [all_pages[i:i+batch_size] for i in range(0, len(all_pages), batch_size)]
    print(f"📦 {len(batches)}개 배치로 분할 (배치당 {batch_size}개)")

    # 3단계: 멀티프로세싱 처리
    print(f"\n⚡ 2단계: 멀티프로세싱 배치 처리")
    all_results = []
    processed_count = 0

    with ProcessPoolExecutor(max_workers=mp.cpu_count()) as executor:
        # 모든 배치를 병렬로 처리
        futures = [executor.submit(process_pages_batch, batch) for batch in batches]

        for future in tqdm(as_completed(futures), total=len(futures), desc="배치 처리"):
            try:
                batch_results = future.result()
                all_results.extend(batch_results)
                processed_count += len(batch_results)

                # 중간 저장
                if processed_count % config['save_interval'] == 0:
                    temp_filename = f"temp_wikisource_{processed_count}.json"
                    with open(temp_filename, 'w', encoding='utf-8') as f:
                        json.dump(all_results, f, ensure_ascii=False, indent=2)
                    print(f"\n💾 중간 저장: {temp_filename} ({processed_count:,}개 페이지)")

            except Exception as e:
                print(f"배치 처리 오류: {e}")
                continue

    print(f"✅ 기본 파싱 완료: {len(all_results):,}개 페이지")

    # 4단계: API 보강 (선택적)
    if config['enable_api'] and all_results:
        print(f"\n🌐 3단계: API 보강 처리")
        enhanced_results = []
        api_success = 0

        for i, page_data in enumerate(tqdm(all_results, desc="API 보강")):
            try:
                enhanced = enhance_with_api(page_data)
                enhanced_results.append(enhanced)

                # API 성공 카운트
                if enhanced.get('api_categories'):
                    api_success += 1

                # API 호출 제한
                time.sleep(config['api_delay'])

                # 중간 저장 (API 보강 버전)
                if (i + 1) % config['save_interval'] == 0:
                    temp_filename = f"temp_wikisource_api_{i+1}.json"
                    with open(temp_filename, 'w', encoding='utf-8') as f:
                        json.dump(enhanced_results, f, ensure_ascii=False, indent=2)
                    print(f"\n💾 API 보강 중간 저장: {temp_filename} ({i+1:,}개 페이지)")

            except Exception as e:
                print(f"API 보강 오류 ({page_data.get('title', 'Unknown')}): {e}")
                enhanced_results.append(page_data)

        all_results = enhanced_results
        print(f"✅ API 보강 완료: {api_success:,}/{len(all_results):,}개 성공 ({api_success/len(all_results)*100:.1f}%)")

    total_time = time.time() - start_time
    print(f"\n🎉 전체 파싱 완료!")
    print(f"  ⏱️  총 시간: {total_time/60:.1f}분")
    print(f"  📊 처리 속도: {len(all_results)/(total_time/60):.1f} 페이지/분")
    print(f"  📄 총 페이지: {len(all_results):,}개")

    return all_results

# 전체 파싱 실행
if EXECUTE_FULL_PARSING:
    print("🔥 전체 위키문헌 파싱을 시작합니다!")
    print("이 작업은 오래 걸릴 수 있습니다. 진행상황을 확인하세요.")

    full_results = parse_full_wikisource(dump_file, FULL_PARSING_CONFIG)

    print(f"\n✅ 전체 파싱이 완료되었습니다!")
    print(f"변수 'full_results'에 {len(full_results):,}개 페이지가 저장되었습니다.")

else:
    print("⏸️  전체 파싱이 비활성화되어 있습니다.")
    print("EXECUTE_FULL_PARSING = True로 설정하고 다시 실행하세요.")

🔥 전체 위키문헌 파싱을 시작합니다!
이 작업은 오래 걸릴 수 있습니다. 진행상황을 확인하세요.
🚀 전체 위키문헌 파싱 시작!
  📊 CPU 코어: 2개 활용
  🌐 API 보강: 사용
  💾 중간 저장: 1000개마다

📥 1단계: 전체 덤프 데이터 수집


전체 페이지 수집: 19825it [01:14, 503.15it/s]

## 📊 전체 파싱 결과 분석

전체 위키문헌 파싱이 완료된 후 상세한 통계와 분석을 확인합니다.

In [17]:
# 전체 파싱 결과 분석
if 'full_results' in locals() and full_results:
    print("📊 전체 위키문헌 파싱 결과 상세 분석")
    print("="*60)

    # 기본 통계
    total_pages = len(full_results)
    api_enhanced = sum(1 for page in full_results if page.get('api_categories'))
    year_found = sum(1 for page in full_results if page.get('year'))
    authors_found = sum(1 for page in full_results if page.get('authors'))
    content_pages = sum(1 for page in full_results if page.get('content_size', 0) > 100)
    substantial_content = sum(1 for page in full_results if page.get('content_size', 0) > 1000)

    print(f"📈 기본 통계:")
    print(f"  총 페이지: {total_pages:,}개")
    print(f"  API 보강된 페이지: {api_enhanced:,}개 ({api_enhanced/total_pages*100:.1f}%)")
    print(f"  연도 정보 있는 페이지: {year_found:,}개 ({year_found/total_pages*100:.1f}%)")
    print(f"  저자 정보 있는 페이지: {authors_found:,}개 ({authors_found/total_pages*100:.1f}%)")
    print(f"  본문 있는 페이지: {content_pages:,}개 ({content_pages/total_pages*100:.1f}%)")
    print(f"  실질적 본문(1000자+): {substantial_content:,}개 ({substantial_content/total_pages*100:.1f}%)")

    # 네임스페이스 분석
    namespace_counts = {}
    for page in full_results:
        ns = page.get('namespace', 0)
        namespace_counts[ns] = namespace_counts.get(ns, 0) + 1

    print(f"\n📂 네임스페이스 분포:")
    for ns, count in sorted(namespace_counts.items()):
        ns_name = {0: '본문', 100: '저자', 102: '색인', 104: '번역'}.get(ns, f'기타({ns})')
        print(f"  {ns_name}: {count:,}개 ({count/total_pages*100:.1f}%)")

    # 분류 분석
    all_categories = []
    for page in full_results:
        all_categories.extend(page.get('categories', []))

    category_counts = {}
    for cat in all_categories:
        category_counts[cat] = category_counts.get(cat, 0) + 1

    top_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)[:20]

    print(f"\n📂 상위 20개 분류:")
    for i, (cat, count) in enumerate(top_categories, 1):
        print(f"  {i:2d}. {cat}: {count:,}개")

    # 연도별 분포
    years = [page.get('year') for page in full_results if page.get('year')]
    if years:
        year_counts = {}
        for year in years:
            try:
                year_int = int(year)
                decade = (year_int // 10) * 10
                year_counts[decade] = year_counts.get(decade, 0) + 1
            except (ValueError, TypeError):
                continue

        print(f"\n📅 연대별 작품 분포:")
        for decade in sorted(year_counts.keys()):
            count = year_counts[decade]
            print(f"  {decade}년대: {count:,}개")

    # 언어 분석
    language_counts = {}
    for page in full_results:
        lang = page.get('language', '미분류')
        language_counts[lang] = language_counts.get(lang, 0) + 1

    print(f"\n🔤 언어 분포:")
    for lang, count in sorted(language_counts.items(), key=lambda x: x[1], reverse=True):
        if lang and count > 0:
            print(f"  {lang}: {count:,}개 ({count/total_pages*100:.1f}%)")

    # 본문 길이 분석
    content_lengths = [page.get('content_size', 0) for page in full_results]
    content_lengths = [x for x in content_lengths if x > 0]

    if content_lengths:
        import statistics
        avg_length = statistics.mean(content_lengths)
        median_length = statistics.median(content_lengths)
        max_length = max(content_lengths)

        print(f"\n📏 본문 길이 분석:")
        print(f"  평균 길이: {avg_length:.0f}자")
        print(f"  중간값: {median_length:.0f}자")
        print(f"  최대 길이: {max_length:,}자")

        # 길이별 분포
        length_ranges = [
            (0, 100, "매우 짧음"),
            (100, 1000, "짧음"),
            (1000, 5000, "중간"),
            (5000, 20000, "긴 편"),
            (20000, float('inf'), "매우 긴")
        ]

        print(f"\n📊 길이별 분포:")
        for min_len, max_len, desc in length_ranges:
            count = sum(1 for x in content_lengths if min_len <= x < max_len)
            if count > 0:
                print(f"  {desc} ({min_len:,}-{max_len:,}자): {count:,}개 ({count/len(content_lengths)*100:.1f}%)")

    # 가장 긴 문서들
    longest_pages = sorted(full_results, key=lambda x: x.get('content_size', 0), reverse=True)[:10]

    print(f"\n📖 가장 긴 문서 Top 10:")
    for i, page in enumerate(longest_pages, 1):
        title = page['title']
        length = page.get('content_size', 0)
        print(f"  {i:2d}. {title}: {length:,}자")

    print(f"\n🎯 품질 지표:")
    print(f"  완전한 메타데이터 페이지: {sum(1 for p in full_results if p.get('authors') and p.get('categories') and p.get('year')):,}개")
    print(f"  API 보강 성공률: {api_enhanced/total_pages*100:.1f}%")
    print(f"  유의미한 본문 비율: {content_pages/total_pages*100:.1f}%")
    print(f"  구조화된 정보 비율: {year_found/total_pages*100:.1f}%")

else:
    print("❌ 전체 파싱 결과가 없습니다.")
    print("💡 먼저 전체 파싱을 실행하세요.")


❌ 전체 파싱 결과가 없습니다.
💡 먼저 전체 파싱을 실행하세요.


## 💾 전체 데이터셋 최종 저장

전체 위키문헌 파싱 결과를 완전한 데이터셋으로 저장합니다.

## 🎓 튜토리얼 완료!

### 🎉 축하합니다! 다음을 배웠습니다:

1. **위키문헌 XML 덤프 파싱**
   - 위키텍스트에서 메타데이터 추출
   - 마크업 제거하여 깔끔한 본문 생성
   - 멀티프로세싱으로 빠른 처리

2. **API 보강 기법**
   - 위키문헌 API로 완전한 분류 정보 추출
   - 위키데이터 API로 구조화된 연도 정보 획득
   - 여러 소스 정보 통합 및 검증

3. **실제 문제 해결**
   - 애국가 사례: 2개 → 4개 분류 완전 추출
   - "1941년 작품" 분류에서 연도 정보 추출
   - 템플릿 기반 분류(PD-old-50) 추출
   - 애국가 가사(한자+한글) 완전 보존 확인

4. **대량 처리 및 데이터 저장**
   - 멀티프로세싱으로 대량 페이지 처리
   - JSON, CSV 다중 형식 저장
   - API 보강 효과 추적 및 분석
   - 재사용 가능한 데이터셋 생성

5. **전체 위키문헌 파싱 (고급)**
   - 수천 개 페이지의 완전한 데이터셋 생성
   - 중간 저장으로 안정성 보장
   - 연구용, 분석용, 텍스트용 다중 형식
   - 완전한 메타데이터와 통계 정보

### 🚀 다음 단계:
- **소규모 연구**: 대량 처리 결과로 특정 주제 분석
- **대규모 연구**: 전체 위키문헌 데이터셋으로 종합 분석
- **응용 개발**: 추출된 데이터로 검색 엔진, 추천 시스템 구축
- **다른 프로젝트**: 위키백과, 위키낱말사전 등으로 확장

### 💡 핵심 교훈:
- **단일 소스의 한계**: XML 덤프만으로는 완전한 정보 추출 어려움
- **API 보강의 중요성**: 웹사이트와 동일한 완전한 데이터 획득
- **검증의 필요성**: 여러 소스에서 정보를 교차 확인
- **성능 최적화**: 멀티프로세싱으로 대량 데이터 효율적 처리
- **데이터 품질**: 본문 내용까지 완전히 보존하는 파싱
- **확장성 고려**: 중간 저장과 배치 처리로 대규모 데이터 안정적 처리

### 📊 생성 가능한 데이터:
- **완전한 JSON**: 모든 메타데이터와 본문 포함
- **연구용 CSV**: 핵심 정보만 정리된 분석용 데이터
- **상세 CSV**: API 보강 과정까지 추적 가능한 상세 데이터
- **텍스트 CSV**: 본문 분석 전용 데이터
- **메타데이터**: 데이터셋 통계와 품질 정보

**이제 여러분도 위키문헌 데이터 과학자입니다! 🌟**

---

### 🔄 실행 가이드:

**📝 기본 학습 (권장):**
1. "기본 파싱" → "API 보강 파싱" → "비교 분석" 순서로 실행
2. 애국가 사례로 핵심 개념 이해
3. 소규모 대량 처리(10-50개)로 경험 쌓기

**🚀 대량 처리:**
1. `BULK_LIMIT` 값을 100-1000으로 설정
2. 결과 분석 및 저장 실행
3. 생성된 CSV/JSON으로 분석 실습

**🔥 전체 파싱 (고급):**
1. `EXECUTE_FULL_PARSING = True` 설정
2. 수십 분-몇 시간 대기 (진행상황 모니터링)
3. 완전한 한국어 위키문헌 데이터셋 획득
4. 연구 프로젝트나 논문에 활용

**⚠️ 주의사항:**
- 전체 파싱은 많은 시간과 리소스 필요
- Colab Pro 권장 (더 많은 메모리와 시간)
- API 호출 제한으로 인한 지연 가능